In [99]:
import pandas as pd
import requests
from itertools import combinations

In [100]:
api_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    print("Succesfully fetched data!")
else:
    print(f"Error while fetching data. Status code: {response.status_code}")

Succesfully fetched data!


In [101]:
print(data.keys())

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])


In [102]:
players = pd.DataFrame(data['elements'])
players.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,138,83,674,304,682,310,636,278,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-2,2,0,...,32,15,303,38,459,49,214,37,0.74,0.37
2,True,True,75.0,75.0,226597,0,0,2,-2,2,...,66,3,76,17,31,4,13,5,1.08,0.33
3,True,True,100.0,100.0,219847,0,0,-1,1,1,...,13,5,88,18,37,14,37,10,1.01,0.29
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,638,63,425,51,514,58,558,68,0.00,0.00


In [103]:
print("Players DataFrame Columns:")
print(players.columns.tolist())


Players DataFrame Columns:
['can_transact', 'can_select', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'removed', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'region', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'influence_

In [104]:
position_map = {
    1: 'GK',
    2: 'Defender',
    3: 'Midfielder',
    4: 'Forward'
}

players_selected = players[
    ['first_name', 'second_name', 'team_code', 'element_type', 'points_per_game', 'now_cost']
].copy()

players_selected['now_cost'] = players_selected['now_cost'] / 10

players_selected['element_type'] = players_selected['element_type'].map(position_map)

players_selected[['points_per_game', 'now_cost']] = players_selected[
    ['points_per_game', 'now_cost']
].apply(pd.to_numeric, errors='coerce')

players_selected = players_selected.loc[players_selected['points_per_game'] != 0.0]

players_selected['full_name'] = players_selected['first_name'] + ' ' + players_selected['second_name']

print(players_selected)


         first_name             second_name  team_code element_type  \
1           Gabriel       Fernando de Jesus          3      Forward   
2           Gabriel    dos Santos Magalhães          3     Defender   
3               Kai                 Havertz          3      Forward   
5           Jurriën                  Timber          3     Defender   
6        Jorge Luiz            Frello Filho          3   Midfielder   
..              ...                     ...        ...          ...   
686          Jørgen           Strand Larsen         39      Forward   
687    Toti António                   Gomes         39     Defender   
689           André  Trindade da Costa Neto         39   Midfielder   
690  Carlos Roberto            Forbs Borges         39   Midfielder   
691           Alfie                    Pond         39     Defender   

     points_per_game  now_cost                     full_name  
1                0.8       6.8     Gabriel Fernando de Jesus  
2                4.6 

In [105]:
max_indeces = players_selected[['points_per_game', 'now_cost']].idxmax()
max_rows = players_selected.loc[max_indeces]
print(max_rows)


    first_name second_name  team_code element_type  points_per_game  now_cost  \
401    Mohamed       Salah         14   Midfielder             10.8      13.3   
428     Erling     Haaland         43      Forward              6.5      14.9   

          full_name  
401   Mohamed Salah  
428  Erling Haaland  


In [106]:
goalkeepers = players_selected[players_selected['element_type'] == 'GK']
defenders = players_selected[players_selected['element_type'] == 'Defender']
midfielders = players_selected[players_selected['element_type'] == 'Midfielder']
forwards = players_selected[players_selected['element_type'] == 'Forward']

print(goalkeepers)

    first_name        second_name  team_code element_type  points_per_game  \
11       David        Raya Martin          3           GK              3.9   
21    Norberto        Murara Neto          3           GK              3.5   
52    Emiliano    Martínez Romero          7           GK              2.9   
56       Robin              Olsen          7           GK              1.0   
94        Mark            Travers         91           GK              3.3   
96        Kepa       Arrizabalaga         91           GK              3.1   
108       Mark            Flekken         94           GK              3.1   
165      Jason             Steele         36           GK              4.5   
169       Bart         Verbruggen         36           GK              2.7   
216     Robert            Sánchez          8           GK              3.7   
222      Filip          Jørgensen          8           GK              3.0   
234       Dean          Henderson         31           GK       

In [107]:
print(f"Total GK: {len(goalkeepers)}")
print(f"Total DEF: {len(defenders)}")
print(f"Total MID: {len(midfielders)}")
print(f"Total FWD: {len(forwards)}")

Total GK: 34
Total DEF: 161
Total MID: 229
Total FWD: 51


In [108]:
min_indeces = goalkeepers[['points_per_game', 'now_cost']].idxmin()
min_rows = goalkeepers.loc[min_indeces]
print(min_rows)

min_indeces = defenders[['points_per_game', 'now_cost']].idxmin()
min_rows = defenders.loc[min_indeces]
print(min_rows)

min_indeces = midfielders[['points_per_game', 'now_cost']].idxmin()
min_rows = midfielders.loc[min_indeces]
print(min_rows)

min_indeces = forwards[['points_per_game', 'now_cost']].idxmin()
min_rows = forwards.loc[min_indeces]
print(min_rows)

    first_name second_name  team_code element_type  points_per_game  now_cost  \
56       Robin       Olsen          7           GK              1.0       4.5   
413  Vítezslav       Jaros         14           GK              1.0       4.0   

           full_name  
56       Robin Olsen  
413  Vítezslav Jaros  
    first_name second_name  team_code element_type  points_per_game  now_cost  \
324      Harry      Clarke         40     Defender             -0.2       4.0   
80       James        Hill         91     Defender              1.0       3.9   

        full_name  
324  Harry Clarke  
80     James Hill  
    first_name   second_name  team_code element_type  points_per_game  \
24       Myles  Lewis-Skelly          3   Midfielder              0.5   
356      Hamza     Choudhury         13   Midfielder              1.0   

     now_cost           full_name  
24        4.5  Myles Lewis-Skelly  
356       4.3     Hamza Choudhury  
    first_name second_name  team_code element_type  poi

In [109]:
TOP_GK = 5     # Number of Goalkeepers
TOP_DEF = 10     # Number of Defenders
TOP_MID = 10     # Number of Midfielders
TOP_FWD = 10     # Number of Forwards

# Select top candidates based on points_per_game
top_goalkeepers = goalkeepers.nlargest(TOP_GK, 'points_per_game').reset_index(drop=True)
top_defenders = defenders.nlargest(TOP_DEF, 'points_per_game').reset_index(drop=True)
top_midfielders = midfielders.nlargest(TOP_MID, 'points_per_game').reset_index(drop=True)
top_forwards = forwards.nlargest(TOP_FWD, 'points_per_game').reset_index(drop=True)

In [110]:
# Generate all possible combinations for each position
gk_combinations = list(combinations(top_goalkeepers['full_name'], 2))      # 2 Goalkeepers
def_combinations = list(combinations(top_defenders['full_name'], 3))      # 5 Defenders
mid_combinations = list(combinations(top_midfielders['full_name'], 4))    # 5 Midfielders
fwd_combinations = list(combinations(top_forwards['full_name'], 3))       # 3 Forwards

print(f"Total GK combinations: {len(gk_combinations)}")
print(f"Total DEF combinations: {len(def_combinations)}")
print(f"Total MID combinations: {len(mid_combinations)}")
print(f"Total FWD combinations: {len(fwd_combinations)}")


Total GK combinations: 5
Total DEF combinations: 120
Total MID combinations: 210
Total FWD combinations: 120


In [111]:
# Combine all top players into a single DataFrame for easy lookup
all_top_players = pd.concat([top_goalkeepers, top_defenders, top_midfielders, top_forwards], ignore_index=True)

# Create a dictionary for quick access to player details
player_lookup = all_top_players.set_index('full_name').to_dict('index')

# Initialize variables to track the optimal team
optimal_team = None
max_total_points = 0
best_cost = 0

# Iterate through all possible combinations
for gk in gk_combinations:
    for def_ in def_combinations:
        for mid in mid_combinations:
            for fwd in fwd_combinations:
                # Combine all selected players
                team_players = list(gk) + list(def_) + list(mid) + list(fwd)
                
                # Calculate total cost
                total_cost = sum([player_lookup[player]['now_cost'] for player in team_players])
                
                # Check budget constraint
                if total_cost > 86.4:
                    continue  # Skip teams that exceed the budget
                
                # Check maximum players from a single real-life team
                team_codes = [player_lookup[player]['team_code'] for player in team_players]
                team_counts = pd.Series(team_codes).value_counts()
                if any(team_counts > 3):
                    continue  # Skip teams that violate team constraints
                
                # Calculate total points per game
                total_points = sum([player_lookup[player]['points_per_game'] for player in team_players])
                
                # Update optimal team if current team has higher points
                if total_points > max_total_points:
                    max_total_points = total_points
                    optimal_team = team_players
                    best_cost = total_cost



In [112]:
# Display the optimal team
if optimal_team:
    print(f"\nOptimal Team Total Points Per Game: {max_total_points}")
    print(f"\nOptimal Team Total Cost: {best_cost}")
    print("Selected Players:")
    for player in optimal_team:
        info = player_lookup[player]
        print(f"{player} - {info['element_type']} - Team Code: {info['team_code']} - Points/Game: {info['points_per_game']} - Cost: £{info['now_cost']}m")
else:
    print("No valid team found within the constraints.")



Optimal Team Total Points Per Game: 70.9

Optimal Team Total Cost: 82.30000000000001
Selected Players:
Alisson Ramses Becker - GK - Team Code: 14 - Points/Game: 5.8 - Cost: £5.4m
Axel Disasi - Defender - Team Code: 8 - Points/Game: 5.0 - Cost: £4.2m
Ashley Young - Defender - Team Code: 11 - Points/Game: 4.8 - Cost: £4.6m
Trent Alexander-Arnold - Defender - Team Code: 14 - Points/Game: 4.7 - Cost: £7.0m
Mohamed Salah - Midfielder - Team Code: 14 - Points/Game: 10.8 - Cost: £13.3m
Cole Palmer - Midfielder - Team Code: 8 - Points/Game: 8.3 - Cost: £11.1m
Bukayo Saka - Midfielder - Team Code: 3 - Points/Game: 7.4 - Cost: £10.5m
Bryan Mbeumo - Midfielder - Team Code: 94 - Points/Game: 6.4 - Cost: £7.6m
Yoane Wissa - Forward - Team Code: 94 - Points/Game: 6.1 - Cost: £6.2m
Chris Wood - Forward - Team Code: 17 - Points/Game: 5.9 - Cost: £6.5m
João Pedro Junqueira de Jesus - Forward - Team Code: 36 - Points/Game: 5.7 - Cost: £5.9m
